# Compute perturbagen signatures from gold signature consensi

In [ ]:
import gzip

import pandas
import sqlite3

import l1000

## Read probe information

In [ ]:
# construct gene_df
probe_df = pandas.read_table('data/geneinfo/geneinfo.tsv.gz')
probe_to_gene = dict(zip(probe_df.pr_id, probe_df.pr_gene_id))
probe_df = probe_df[probe_df.pr_pool_id.str.startswith('epsilon').fillna(False)]
print(len(probe_df))
probe_df.head(2)

# Preparations

In [ ]:
def run_consensi(sig_expr_df, pert_to_sigs, name):
    """Compute consensi signatures"""
    pert_expr_df = l1000.get_consensus_signatures(sig_expr_df, pert_to_sigs)
    pert_expr_df = l1000.probes_to_genes(pert_expr_df, probe_to_gene)
    pert_expr_df = pert_expr_df.transpose()
    pert_expr_df.index.name = 'perturbagen'
    print(pert_expr_df.shape)
    path = 'data/consensi/consensi-{}.tsv.gz'.format(name)
    with gzip.open(path, "w") as write_file:
        pert_expr_df.reset_index().to_csv(write_file, sep='\t', index=False, float_format='%.3f')
    return pert_expr_df

In [ ]:
# open database connection
connection = sqlite3.connect('data/l1000.db')

In [ ]:
# get all gold signatures
query = "SELECT sigs.sig_id FROM sigs WHERE sigs.is_gold = 1"
sigs = pandas.read_sql(query, connection).sig_id.tolist()

# get all epsilon probes
probes = probe_df.pr_id.tolist()

# read expression values
path = 'download/modzs.gctx'
sig_expr_df = l1000.extract_from_gctx(path, probes, sigs)

## consensi for all pert_ids

In [ ]:
query = """
SELECT perts.pert_id, perts.pert_iname, perts.pert_type, sigs.sig_id
FROM perts, sigs
WHERE sigs.pert_id = perts.pert_id
AND sigs.is_gold = 1
"""
sig_df = pandas.read_sql(query, connection)
sig_df.head(2)

In [ ]:
# Condense to perturbagens
pert_to_sigs = {k: g['sig_id'].tolist() for k, g in sig_df.groupby('pert_id')}
pert_expr_df = run_consensi(sig_expr_df, pert_to_sigs, name='all')
pert_expr_df.head()

## drugbank consensi

In [ ]:
query = """
SELECT unichem.resource_id AS drugbank_id, perts.pert_id, sigs.sig_id, sigs.is_gold
FROM unichem, perts, sigs
WHERE unichem.resource = 'drugbank'
AND unichem.pert_uid = perts.pert_uid
AND sigs.pert_id = perts.pert_id
"""
sig_df = pandas.read_sql(query, connection)
sig_df = sig_df.query("is_gold == 1")

In [ ]:
pert_to_sigs = {k: g['sig_id'].tolist() for k, g in sig_df.groupby('drugbank_id')}
pert_expr_df = run_consensi(sig_expr_df, pert_to_sigs, name='drugbank')
pert_expr_df.head()

## knockdown consensi

In [ ]:
query = """
SELECT perts.pert_id, perts.pert_iname, perts.pert_type, sigs.sig_id
FROM perts, sigs
WHERE sigs.pert_id = perts.pert_id
AND pert_type = 'trt_sh'
AND sigs.is_gold = 1
"""
sig_df = pandas.read_sql(query, connection)
sig_df.head(2)

In [ ]:
# Condense to perturbagens
pert_to_sigs = {k: g['sig_id'].tolist() for k, g in sig_df.groupby('pert_iname')}
pert_expr_df = run_consensi(sig_expr_df, pert_to_sigs, name='knockdown')
pert_expr_df.head()

## overexpression consensi

In [ ]:
query = """
SELECT perts.pert_id, perts.pert_iname, perts.pert_type, sigs.sig_id
FROM perts, sigs
WHERE sigs.pert_id = perts.pert_id
AND pert_type = 'trt_oe'
AND sigs.is_gold = 1
"""
sig_df = pandas.read_sql(query, connection)
sig_df.head(2)

In [ ]:
# Condense to perturbagens
pert_to_sigs = {k: g['sig_id'].tolist() for k, g in sig_df.groupby('pert_iname')}
pert_expr_df = run_consensi(sig_expr_df, pert_to_sigs, name='overexpression')
pert_expr_df.head()